# 单个数据预处理

为构建我们的本地知识库，我们需要对以多种类型存储的本地文档进行处理，读取本地文档并通过前文描述的 Embedding 方法将本地文档的内容转化为词向量来构建向量数据库。接下来以一些实际示例入手，来讲解如何对本地文档进行处理。
## 数据读取
将知识库源数据放置在../../data_base/knowledge_db 目录下

**PDF文档为例**  
可以使用 LangChain 的 PyMuPDFLoader 来读取知识库的 PDF 文件。PyMuPDFLoader 是 PDF 解析器中速度最快的一种，结果会包含 PDF 及其页面的详细元数据，并且每页返回一个文档。

In [33]:
from langchain.document_loaders.pdf import PyMuPDFLoader

# 创建一个 PyMuPDFLoader Class 实例，输入为待加载的 pdf 文档路径
loader = PyMuPDFLoader("../data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）安全加固指导.pdf")

# 调用 PyMuPDFLoader Class 的函数 load 对 pdf 文件进行加载
pdf_pages = loader.load()

文档加载后储存在 `pdf_pages` 变量中:
- `pdf_pages` 的变量类型为 `List`
- 打印 `pdf_pages` 的长度可以看到 pdf 一共包含多少页

In [34]:
print(f"载入后的变量类型为：{type(pdf_pages)}，",  f"该 PDF 一共包含 {len(pdf_pages)} 页")

载入后的变量类型为：<class 'list'>， 该 PDF 一共包含 25 页


`pdf_pages` 中的每一元素为一个文档，变量类型为 `langchain_core.documents.base.Document`, 文档变量类型包含两个属性
- `page_content` 包含该文档的内容。
- `meta_data` 为文档相关的描述性数据。

In [35]:
pdf_page = pdf_pages[2]
print(f"每一个元素的类型：{type(pdf_page)}.", 
    f"该文档的描述性数据：{pdf_page.metadata}", 
    f"查看该文档的内容:\n{pdf_page.page_content}", 
    sep="\n------\n")

每一个元素的类型：<class 'langchain_core.documents.base.Document'>.
------
该文档的描述性数据：{'producer': 'Qt 4.8.7', 'creator': 'wkhtmltopdf 0.12.6', 'creationdate': '2024-11-07T01:28:31+00:00', 'source': '../data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）安全加固指导.pdf', 'file_path': '../data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）安全加固指导.pdf', 'total_pages': 25, 'format': 'PDF 1.4', 'title': '安全加固指导', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2024-11-07T09:33:23+08:00', 'trapped': '', 'modDate': "D:20241107093323+08'00'", 'creationDate': 'D:20241107012831Z', 'page': 2}
------
查看该文档的内容:
11.. 适
适⽤
⽤范
范围
围
产品安全加固的⽬的是“通过配置、设置、协议限制来减⼩攻击⾯”（来源于《产品安全要求总则》），使得产品免于或减⼩所受⽹络安全攻击的影响。
产品安全加固对象包括产品所使⽤的第三⽅软件（如操作系统等），以及部分⾃研模块。
安全合加固置是通过对加固对象中安全相关的配置项进⾏合理的设置，减⼩对象的攻击⾯，例如关闭不必要的端⼝、不必要的系统服务等；安全漏洞补丁是通过对加固对象的漏洞缺陷
通过补丁升级的⽅式，使加固对象免予或减⼩遭受该漏洞带来的攻击影响，例如各类CVE漏洞、CNVD漏洞的补丁升级处理等。
本⽂档⽤于规范和指导产品安全加固配置⼯作，包括产品所使⽤的第三⽅软件（如操作系统等），以及部分⾃研模块的安全加固配置。
本⽂档适⽤于VMAX产品的安全加固配置⼯作。 适⽤于该产品所有的产品软件版本。
适⽤于CGSLV5或V6操作系统及国产化⻰晰操作系统。
使⽤模板：JF-051 安全

In [36]:
# 假设环境中缺少 NLTK 数据包中的 punkt 资源，这是 NLTK 库中用于分词的一个重要组件。解决这个问题的方法是按照报错信息中提到的步骤下载并安装 punkt 资源。
# import nltk
# nltk.download('punkt', quiet=True)

## 数据清洗

我们期望知识库的数据尽量是有序的、优质的、精简的，因此我们要删除低质量的、甚至影响理解的文本数据。  
可以看到上文中读取的pdf文件不仅将一句话按照原文的分行添加了换行符`\n`，也在原本两个符号中间插入了`\n`，我们可以使用正则表达式匹配并删除掉`\n`。

**处理单个page**

In [37]:
import re

# 预编译正则表达式（提升效率）
linebreak_pattern = re.compile(
    r'(?<![\\u4e00-\\u9fff])\n(?![\\u4e00-\\u9fff])',  # 负向断言匹配非中文环境换行
    flags=re.DOTALL
)
space_pattern = re.compile(r'[ 　]+')  # 匹配半角/全角空格
special_chars = ['•', '▪', '▫', '▶', '®', '©']  # 可扩展的干扰符号列表

In [38]:
pdf_page.page_content = re.sub(
        linebreak_pattern,
        lambda m: m.group().replace('\n', ''),
        pdf_page.page_content
    )
    
# 2. 批量清理特殊符号
for char in special_chars:
        pdf_page.page_content = pdf_page.page_content.replace(char, '')
    
# 3. 安全删除空格（保留URL等特殊场景）
pdf_page.page_content = space_pattern.sub('', pdf_page.page_content)

pdf_page.page_content

'11..适适⽤⽤范范围围产品安全加固的⽬的是“通过配置、设置、协议限制来减⼩攻击⾯”（来源于《产品安全要求总则》），使得产品免于或减⼩所受⽹络安全攻击的影响。产品安全加固对象包括产品所使⽤的第三⽅软件（如操作系统等），以及部分⾃研模块。安全合加固置是通过对加固对象中安全相关的配置项进⾏合理的设置，减⼩对象的攻击⾯，例如关闭不必要的端⼝、不必要的系统服务等；安全漏洞补丁是通过对加固对象的漏洞缺陷通过补丁升级的⽅式，使加固对象免予或减⼩遭受该漏洞带来的攻击影响，例如各类CVE漏洞、CNVD漏洞的补丁升级处理等。本⽂档⽤于规范和指导产品安全加固配置⼯作，包括产品所使⽤的第三⽅软件（如操作系统等），以及部分⾃研模块的安全加固配置。本⽂档适⽤于VMAX产品的安全加固配置⼯作。适⽤于该产品所有的产品软件版本。适⽤于CGSLV5或V6操作系统及国产化⻰晰操作系统。使⽤模板：JF-051安全加固指南模板R1.3\n22..概概述述\n22..11系系统统安安全全架架构构模模型型\nVMAX的安全体系以数据为中⼼，重点考虑数据⽣命周期各阶段中的数据安全问题，形成了统⼀的安全框架，通过在数据全⽣命周期各环节实施安全技术和管理机制，为系统及⽤⼾数据提供安全保障。\nVMAX的安全体系在VMAX⾃⾝的安全体系基础上，结合⼤数据平台的安全体系，形成了以数据安全机制为核⼼的数据安全架构，安全架构模型如下图所⽰：\n1)环境安全层：指VMAX系统安全运⾏所依赖的底层运⾏环境安全，包括操作系统安全、主机安全、⽹络安全、以及基础设施安全等。\n2)⼤数据平台安全层：VMAX的部分功能运⾏于⼤数据平台之上，因此⼤数据平台的安全性是VMAX安全性的⼀部分，包括：Ø接⼊控制：关注于控制外部⽤⼾或者服务对集群的访问过程中的⾝份鉴别，包括⽤⼾账⼾管理模块及⽤⼾⾝份认证模块，这是实施⼤数据安全架构的基础。在访问启⽤了群时，必须能通过服务所需要的安全认证⽅式；Ø访问控制：关注于⽤⼾或者应⽤访问数据时，对⽤⼾的权限定义和实施过程，称为授权模块。访问控制可以限定⽤⼾是否有对某种资源的访问能⼒，能给不同应⽤提供⼀度的访问控制能⼒；Ø合规审计：审计的⽬的是捕获系统内的完整活动记录，且不可被更改，为⽤⼾提供安全事件的事后追溯、定位问题原因及划分事故责任的重要⼿段。\n3)VMAX安全层：利⽤数据加密、传输安全(使⽤标准的

## 文档分割

由于单个文档的长度往往会超过模型支持的上下文，导致检索得到的知识太长超出模型的处理能力，因此，在构建向量知识库的过程中，我们往往需要对文档进行分割，将单个文档按长度或者按固定的规则分割成若干个 chunk，然后将每个 chunk 转化为词向量，存储到向量数据库中。

在检索时，我们会以 chunk 作为检索的元单位，也就是每一次检索到 k 个 chunk 作为模型可以参考来回答用户问题的知识，这个 k 是我们可以自由设定的。

Langchain 中文本分割器都根据 `chunk_size` (块大小)和 `chunk_overlap` (块与块之间的重叠大小)进行分割。

![image.png](../../figures/C3-3-example-splitter.png)

* chunk_size 指每个块包含的字符或 Token （如单词、句子等）的数量

* chunk_overlap 指两个块之间共享的字符数量，用于保持上下文的连贯性，避免分割丢失上下文信息

Langchain 提供多种文档分割方式，区别在怎么确定块与块之间的边界、块由哪些字符/token组成、以及如何测量块大小

- RecursiveCharacterTextSplitter(): 按字符串分割文本，递归地尝试按不同的分隔符进行分割文本。
- CharacterTextSplitter(): 按字符来分割文本。
- MarkdownHeaderTextSplitter(): 基于指定的标题来分割markdown 文件。
- TokenTextSplitter(): 按token来分割文本。
- SentenceTransformersTokenTextSplitter(): 按token来分割文本
- Language(): 用于 CPP、Python、Ruby、Markdown 等。
- NLTKTextSplitter(): 使用 NLTK（自然语言工具包）按句子分割文本。
- SpacyTextSplitter(): 使用 Spacy按句子的切割文本。

In [39]:
''' 
* RecursiveCharacterTextSplitter 递归字符文本分割
RecursiveCharacterTextSplitter 将按不同的字符递归地分割(按照这个优先级["\n\n", "\n", " ", ""])，
    这样就能尽量把所有和语义相关的内容尽可能长时间地保留在同一位置
RecursiveCharacterTextSplitter需要关注的是4个参数：

* separators - 分隔符字符串数组
* chunk_size - 每个文档的字符数量限制
* chunk_overlap - 两份文档重叠区域的长度
* length_function - 长度计算函数
'''
#导入文本分割器
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [40]:
# 知识库中单段文本长度
CHUNK_SIZE = 512

# 知识库中相邻文本重合长度
OVERLAP_SIZE = 50

In [41]:
# 使用递归字符文本分割器
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=OVERLAP_SIZE
)
# text_splitter.split_text(pdf_page.page_content[0:1000])

In [42]:
split_docs = text_splitter.split_documents(pdf_pages)
print(f"切分后的文件数量：{len(split_docs)}")

切分后的文件数量：97


In [43]:
split_docs[2].page_content

'11..适适⽤⽤范范围围产品安全加固的⽬的是“通过配置、设置、协议限制来减⼩攻击⾯”（来源于《产品安全要求总则》），使得产品免于或减⼩所受⽹络安全攻击的影响。产品安全加固对象包括产品所使⽤的第三⽅软件（如操作系统等），以及部分⾃研模块。安全合加固置是通过对加固对象中安全相关的配置项进⾏合理的设置，减⼩对象的攻击⾯，例如关闭不必要的端⼝、不必要的系统服务等；安全漏洞补丁是通过对加固对象的漏洞缺陷通过补丁升级的⽅式，使加固对象免予或减⼩遭受该漏洞带来的攻击影响，例如各类CVE漏洞、CNVD漏洞的补丁升级处理等。本⽂档⽤于规范和指导产品安全加固配置⼯作，包括产品所使⽤的第三⽅软件（如操作系统等），以及部分⾃研模块的安全加固配置。本⽂档适⽤于VMAX产品的安全加固配置⼯作。适⽤于该产品所有的产品软件版本。适⽤于CGSLV5或V6操作系统及国产化⻰晰操作系统。使⽤模板：JF-051安全加固指南模板R1.3\n22..概概述述\n22..11系系统统安安全全架架构构模模型型'

In [44]:
print(f"切分后的字符数（可以用来大致评估 token 数）：{sum([len(doc.page_content) for doc in split_docs])}")

切分后的字符数（可以用来大致评估 token 数）：41859


注：如何对文档进行分割，其实是数据处理中最核心的一步，其往往决定了检索系统的下限。但是，如何选择分割方式，往往具有很强的业务相关性——针对不同的业务、不同的源数据，往往需要设定个性化的文档分割方式。因此，在本章，我们仅简单根据 chunk_size 对文档进行分割。对于有兴趣进一步探索的读者，欢迎阅读我们第三部分的项目示例来参考已有的项目是如何进行文档分割的。

# 批量数据预处理

批量处理文件夹中所有文件

In [22]:
import os

# 获取folder_path下所有文件路径，储存在file_paths里
file_paths = []
folder_path = '../data_base/knowledge_path/VMAX-S'
for root, dirs, files in os.walk(folder_path):
    for file in files:
        file_path = os.path.join(root, file)
        file_paths.append(file_path)
print(file_paths)

from langchain.document_loaders.pdf import PyMuPDFLoader
# from langchain.document_loaders.markdown import UnstructuredMarkdownLoader

# 遍历文件路径并把实例化的loader存放在loaders里
loaders = []

for file_path in file_paths:
    file_type = file_path.split('.')[-1]
    if file_type == 'pdf':
        loaders.append(PyMuPDFLoader(file_path))
    else:
        print(f"Unsupported file type: {file_type} for file {file_path}")


# 下载文件并存储到text
# 加载所有文档内容到 texts
texts = []
for loader in loaders:
    texts.extend(loader.load())  # 关键步骤：初始化 texts


# 作数据清洗
# 修改后的数据清洗部分（替换原始代码中对应段落）
import re

# 预编译正则表达式（提升效率）
linebreak_pattern = re.compile(
    r'(?<![\\u4e00-\\u9fff])\n(?![\\u4e00-\\u9fff])',  # 负向断言匹配非中文环境换行
    flags=re.DOTALL
)
space_pattern = re.compile(r'[ 　]+')  # 匹配半角/全角空格
special_chars = ['•', '▪', '▫', '▶', '®', '©']  # 可扩展的干扰符号列表

# 替换原始代码中的清洗循环
for text in texts:
    # 1. 清理非中文环境换行
    text.page_content = re.sub(
        linebreak_pattern,
        lambda m: m.group().replace('\n', ''),
        text.page_content
    )
    
    # 2. 批量清理特殊符号
    for char in special_chars:
        text.page_content = text.page_content.replace(char, '')
    
    # 3. 安全删除空格（保留URL等特殊场景）
    text.page_content = space_pattern.sub('', text.page_content)

['../data_base/knowledge_path/VMAX-S\\SJ-20130403110434-005-ZXCLOUD E9000（V1.0）刀片服务器 快速安装指南.pdf', '../data_base/knowledge_path/VMAX-S\\SJ-20140307103120-034-ZXCLOUD E9000（V1.0）刀片服务器 硬件安装指导.pdf', '../data_base/knowledge_path/VMAX-S\\SJ-20140326173355-003-ZXCLOUD I8350 G2（V1.0）机架服务器 硬件安装指导.pdf', '../data_base/knowledge_path/VMAX-S\\SJ-20151221160747-002-ZXVMAX（V6.15）多维价值分析系统 软件安装指导.pdf', '../data_base/knowledge_path/VMAX-S\\SJ-20151221160747-011-ZXVMAX（V6.15）多维价值分析系统 软件安装指导（CDMA分册）.pdf', '../data_base/knowledge_path/VMAX-S\\SJ-20151225170610-002-ZXVMAX（V6.15）多维价值分析系统 主题表安装指南（LTE分册）.pdf', '../data_base/knowledge_path/VMAX-S\\ZXVMAX-S（V6.19.20.10.03）硬件安装指导.pdf', '../data_base/knowledge_path/VMAX-S\\ZXVMAX-S（V6.23）安全加固指导.pdf']


In [23]:
# from dotenv import load_dotenv, find_dotenv 
# # pip install python-dotenv

# _ = load_dotenv(find_dotenv())

# from langchain_community.embeddings import ZhipuAIEmbeddings

# zhipu_embed = ZhipuAIEmbeddings(
#     model="embedding-2",
#     api_key="5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io",
# )

In [24]:
''' 
* RecursiveCharacterTextSplitter 递归字符文本分割
RecursiveCharacterTextSplitter 将按不同的字符递归地分割(按照这个优先级["\n\n", "\n", " ", ""])，
    这样就能尽量把所有和语义相关的内容尽可能长时间地保留在同一位置
RecursiveCharacterTextSplitter需要关注的是4个参数：

* separators - 分隔符字符串数组
* chunk_size - 每个文档的字符数量限制
* chunk_overlap - 两份文档重叠区域的长度
* length_function - 长度计算函数
'''
#导入文本分割器
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [25]:
# 知识库中单段文本长度
CHUNK_SIZE = 512

# 知识库中相邻文本重合长度
OVERLAP_SIZE = 50

In [26]:
# 使用递归字符文本分割器
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=OVERLAP_SIZE
)
# text_splitter.split_text(text.page_content[0:1000])

In [27]:
split_docs = text_splitter.split_documents(texts)
print(f"切分后的文件数量：{len(split_docs)}")

切分后的文件数量：695


In [28]:
print(f"切分后的字符数（可以用来大致评估 token 数）：{sum([len(doc.page_content) for doc in split_docs])}")

切分后的字符数（可以用来大致评估 token 数）：236550


In [29]:
split_docs[100].page_content

'6系统上电指指指示示示灯灯灯颜颜颜色色色说说说明明明硬盘指示灯绿色/红色l上面灯为硬盘在位指示灯，绿色常亮：硬盘在位。l下面灯为硬盘工作状态指示灯，绿色闪烁：硬盘正有数据读写。l硬盘工作状态指示灯，红色闪烁：硬盘异常。\nUID指示灯蓝色l蓝色常亮：正在对系统定位操作。l蓝色1Hz闪烁：正在远程操作系统或系统固件升级。l蓝色灭：未对系统定位操作。------步步步骤骤骤结结结束束束------\nSJ-20140326173355-003|2014-04-01（R1.0）\n6-3'